## ハンズオン03: LLM as a Judge

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

ハンズオンに必要な環境変数を読み込みます。

In [8]:
import os, warnings

# おまじない
warnings.filterwarnings("ignore")

Langfuseのクライアントを初期化します。

In [9]:
from langfuse import Langfuse

langfuse = Langfuse()

### 出力に対する評価

LLM as a Judgeの対象となる生成結果の一覧を取得します。今回は、現在時から24時間以内に生成された生成結果を評価対象として扱います。

In [10]:
import datetime
from pprint import pprint

generations = langfuse.get_generations(
    from_start_time=datetime.datetime.now() - datetime.timedelta(hours=24),
)

pprint(f"Fetched {len(generations.data)} generations.")
pprint(f"{generations.data[0].__dict__}")

'Fetched 4 generations.'
("{'id': 'f3a44cfc-cf2b-48fa-bd0e-80a69a0b0e78', 'trace_id': "
 "'80559a08-2e57-4628-ae25-cc18d0ab3096', 'type': 'GENERATION', 'name': "
 "'ChatOpenAI', 'start_time': datetime.datetime(2024, 10, 28, 14, 15, 53, "
 "973000, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2024, "
 '10, 28, 14, 15, 56, 434000, tzinfo=datetime.timezone.utc), '
 "'completion_start_time': None, 'model': 'gpt-4o-mini', 'model_parameters': "
 "{'max_tokens': 1024, 'temperature': '0.7'}, 'input': [{'role': 'user', "
 '\'content\': "\\n以下のコンテキストに基づいて質問に対する回答をBBっぽく作成してください。\\n\\n## '
 'BBとは？\\nBBは、昼はソフトウェアエンジニアで、夜はDJ/VJと二刀流を実現している男性です。\\nそのため、いつ寝ているかわからず常に寝不足です。\\nまた、食事はラーメンや焼肉を中心に取っています。\\nそして、料理を趣味としており数多くの自慢のレシピを持っています。\\n\\n## '
 "コンテキスト\\n[Document(metadata={'source': './docs/butakimuchi.txt', "
 "'relevance_score': 0.050705366}, "
 "page_content='BB流！安くて早くて旨い男のための豚キムチの作り方\\\\n\\\\n材料は以下の通りです。\\\\n\\\\n- "
 '豚ロース薄切り100g\\\\n- キムチ100g\\\\n- タマネギ1/2個（レタス少量などでも可）\\\\n- ごま油\

評価用の関数を実装します。今回は、LangChainのEvaluatorを使用します。

In [ ]:
from langchain.evaluation.loading import load_evaluator
from langchain.evaluation.schema import EvaluatorType

def load_evaluator_by_criteria_key(key: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")

    evaluator = load_evaluator(
        evaluator=EvaluatorType.CRITERIA,
        llm=llm,
        criteria=key
    )
    return evaluator

評価基準を設定します。今回は、

- conciseness: 簡潔で要点をついた回答であるか
- coherence: 構造化され、整理された回答であるか
- harmfulness: 有害、攻撃的、不適切な回答であるか

を評価基準として設定します。

In [ ]:
criterias = [
    "conciseness",
    "coherence",
    "harmfulness",
]

24時間以内の生成結果に対して、実際にLLMによる評価を行います。

In [ ]:
def execute_evaluation_and_scoring():
    for generation in generations.data:
        for key in criterias:
            evaluator = load_evaluator_by_criteria_key(key=key)
            result = evaluator.evaluate_strings(
                prediction=generation.output,
                input=generation.input
            )
            pprint(result)
            langfuse.score(
                name=f"llm-as-a-judge-{key}",
                trace_id=generation.trace_id,
                observation_id=generation.id,
                value=result.get("score"),
                comment=result.get("reasoning")
            )

execute_evaluation_and_scoring()

### （オプション）入力に対する評価

LLM as a Judgeの対象となる一覧を取得します。今回は、現在時から24時間以内に入力されたプロンプトを対象として扱います。

In [11]:
import datetime
from pprint import pprint

traces = langfuse.get_traces(
    from_timestamp=datetime.datetime.now() - datetime.timedelta(hours=24),
    tags=["app"]
)

pprint(f"Fetched {len(traces.data)} generations.")
pprint(f"{traces.data[0].__dict__}")

'Fetched 3 generations.'
("{'id': '3d8567a8-0c27-462c-b198-821db2c03661', 'timestamp': "
 'datetime.datetime(2024, 10, 28, 14, 27, 31, 136000, '
 "tzinfo=datetime.timezone.utc), 'name': 'Ask the BigBaBy', 'input': "
 "'とっても美味しそうなレシピを教えてもらえて嬉しいです！ありがとうございます！', 'output': "
 "'いやいや、こちらこそ嬉しいよ！食べることはやっぱり大事だしね。BB流のレシピは、簡単で美味しいから、ぜひ試してみてほしいな！夜のDJセットの合間にでも作って、ラーメンや焼肉に飽きたときの新しい一品としてピッタリだよ。特にきのこマリネは、お酒とも相性抜群だから、夜のリフレッシュにもなるしね！また何か作りたくなったらいつでも聞いてね！ドンと来い！', "
 "'session_id': '05de9361-173d-4015-b6f5-eda1f669805a', 'release': "
 "'0.0.1-SNAPSHOT', 'version': None, 'user_id': None, 'metadata': None, "
 "'tags': ['app'], 'public': False, 'html_path': "
 "'/project/pj-1234567890/traces/3d8567a8-0c27-462c-b198-821db2c03661', "
 "'latency': 2.3899998664855957, 'total_cost': 0.00027735, 'observations': "
 "['05ac3a10-b602-460c-99b0-0a3f19db7db1', "
 "'3063ef72-5274-41c2-b318-e905819e54b1', "
 "'307e07d7-2305-4eef-9c42-c10a65ad37ba', "
 "'84eb924e-2262-48b2-9792-6776624b7ec0', "
 "'b888df40-19ff-4ce7-960b

評価用の関数を実装します。今回は、ユーザーの入力プロンプトを”否定的”、”中立的”、”肯定的”にLLMを用いて分類を行います。

In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

sentiment_analysis_prompt = """
以下の入力テキストを”否定的”、”中立的”、”肯定的”に分類してください。
また、出力は”否定的”、”中立的”、”肯定的”のみで理由などは含まないでください。

## 入力テキスト

{input}
"""

def sentiment_analysis(input: str) -> str:
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")
    sentiment_analysis_chain = (
        {"input": RunnablePassthrough()}
        | PromptTemplate.from_template(sentiment_analysis_prompt)
        | llm
        | StrOutputParser()
    )
    result = sentiment_analysis_chain.invoke(input)
    return result

入力プロンプトに対する感情分析を実行します。

In [13]:
def execute_sentiment_analysis():
    for trace in traces.data:
        result = sentiment_analysis(input=trace.input)
        score_map = {
            "否定的": 0,
            "中立的": 0.5,
            "肯定的": 1
        }
        pprint({"input": trace.input, "result": result})
        langfuse.score(
            name=f"llm-as-a-judge-sentiment-analysis",
            trace_id=trace.id,
            observation_id=trace.id,
            value=score_map.get(result, 0.5),
            comment=result
        )

execute_sentiment_analysis()

{'input': 'とっても美味しそうなレシピを教えてもらえて嬉しいです！ありがとうございます！', 'result': '肯定的'}
{'input': '男っぽい料理しか作れないんですか？もっと繊細な料理とか紹介してくれるのかと期待していました。残念です。',
 'result': '否定的'}
{'input': '豚肉を使った料理のおすすめを教えてください', 'result': '中立的'}
